In [ ]:
import math
import numpy as np
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.models import load_model
import csv
import os

In [ ]:
# Changeable parameter
TRAIN_DIR = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Train\CV_0'
# TRAIN_DIR = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Augmented10SameClass\CV_0'
VAL_DIR = r'D:\Resources\Inat_Partial\Aves_Small_SS1_Validation\CV_0'
#BATCH_SIZE = 16
# Need smaller batch for deeper convolution
BATCH_SIZE = 8
# Set this to 3 for the large dataset
EPOCH_NUM = 20
# Set this to 0 or 1 for extremely large dataset (e.g. augmented)
CALLBACK_PATIENCE = 20
INPUT_WIDTH = 480
INPUT_HEIGHT = 480
LOAD_PREV_MODEL = True
FINE_TUNE_MODEL = True
# The first layer in the model to unfreeze for the fine tuning
# FINE_TUNE_LAYER = 'conv2d_181'
FINE_TUNE_LAYER = 'conv2d_118'
RANDOM_SEED = 5703
# Make sure to change this to the designated model
PREV_MODEL_PATH = r'inceptionresnetv2_subset1_cv0_shape480_38_epoch.h5'
# PREV_MODEL_PATH = r'inceptionresnetv2_subset1_augmentsame_cv0_shape480_2_epoch.h5'
PREV_FINE_TUNE_PATH = r'inceptionresnetv2_ft2_subset1_cv0_shape480_0_epoch.h5'
MODEL_METRICS_PATH = r'inceptionresnetv2_ft2_subset1_cv0_shape480_history.csv'
# MODEL_METRICS_PATH = r'inceptionresnetv2_ft2_subset1_augmentsame_cv0_shape480_history.csv'

In [ ]:
# Non-changeable parameter
cur_model_path = None

In [ ]:
#Limiting the number of resources used
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [ ]:
# Default configuration from
# https://keras.io/preprocessing/image/
# With a little bit of change in parameter
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# Do not enable extra augmentation for already augmented dataset
# train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(INPUT_HEIGHT, INPUT_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical', 
        seed=RANDOM_SEED)

validation_generator = test_datagen.flow_from_directory(
        VAL_DIR,
        target_size=(INPUT_HEIGHT, INPUT_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=RANDOM_SEED)

In [ ]:
# Not reusing any of the fully connected layer, 
# in case of resnet inception they go straight to the output after the global pooling however
if LOAD_PREV_MODEL == True and FINE_TUNE_MODEL == False and os.path.isfile(PREV_MODEL_PATH) == True:
    print('loading previous model')
    model = load_model(PREV_MODEL_PATH)
elif LOAD_PREV_MODEL == True and FINE_TUNE_MODEL == True and os.path.isfile(PREV_MODEL_PATH) == True:
    # Try to load from the previous fine tuning model, if it does not exist, load the transfer learning model
    print('fine tuning the model')
    if os.path.isfile(PREV_FINE_TUNE_PATH) == True:
        model = load_model(PREV_FINE_TUNE_PATH)
    # Modify the layer freeze state
    else:
        model = load_model(PREV_MODEL_PATH)
        # Unfreeze everything first
        for layer in model.layers:
            layer.trainable = True
        # Then freeze all the layers back until the target layer
        for layer in model.layers:
            if layer.name == FINE_TUNE_LAYER:
                break
            layer.trainable = False
        # Recompile the model with different lower learning rate optimizer
        model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
    # Replace the path to be saved at the latter stage
    PREV_MODEL_PATH = PREV_FINE_TUNE_PATH
else:
    print('no previous model')
    base_model = InceptionResNetV2(include_top=False,weights='imagenet',input_shape=(INPUT_HEIGHT,INPUT_WIDTH,3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # Disable the dense layer
    x = Dense(1024, activation='relu')(x)  
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freezing all the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # and then compile the model
    # Use the default optimizer is fine for the transfer learning stage
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Numbering the current epoch based on the previous epoch path
# Handling multiple re-run of the training cell as well
split_prev_model_path = PREV_MODEL_PATH.split('_') if cur_model_path is None else cur_model_path.split('_')
prev_epoch_number = int(split_prev_model_path[-2])
cur_epoch_number = prev_epoch_number + EPOCH_NUM
cur_model_path = '_'.join(split_prev_model_path[:-2]+[str(cur_epoch_number)]+split_prev_model_path[-1:])

In [ ]:
cur_model_path

In [ ]:
# TRAINING CELL
num_train_samples = train_generator.samples
train_epoch_steps = math.ceil(num_train_samples / BATCH_SIZE)
num_val_samples = validation_generator.samples
val_epoch_steps = math.ceil(num_val_samples / BATCH_SIZE)
# Note: In case of early stopping, rename the saved file accordingly
callbacks = [EarlyStopping(monitor='val_loss', patience=CALLBACK_PATIENCE, verbose=0, mode='auto'), 
             ModelCheckpoint(filepath=cur_model_path, verbose=1, save_best_only=True)]
train_history = model.fit_generator(train_generator,
                    steps_per_epoch=train_epoch_steps,
                    epochs=EPOCH_NUM,
                    validation_data=validation_generator,
                    validation_steps=val_epoch_steps, 
                    callbacks=callbacks)

In [ ]:
# Record the train history in the designated csv file
# Note: newline has to be '' due to the way csv writerow works
with open(MODEL_METRICS_PATH, 'a+', newline='') as history_file:
    csv_writer = csv.writer(history_file)
    for e in train_history.epoch:
        epoch_number = prev_epoch_number + e
        csv_writer.writerow([epoch_number,
                            train_history.history['acc'][e],
                            train_history.history['loss'][e],
                            train_history.history['val_acc'][e],
                            train_history.history['val_loss'][e]])

In [ ]:
# Optionally save the model with the latest epoch result
model.save(cur_model_path)